In [1]:
print ()

import networkx
from operator import itemgetter
import matplotlib.pyplot
import pandas as pd

In [2]:
#Read the data from amazon-books.csv into amazonBooks dataframe;
amazonBooks = pd.read_csv('./amazon-books.csv',index_col=0)

#Read the data from amazon-books-copurchase.adjlist;
#assign it to copurchaseGraph weighted Graph;
#node = ASIN, edge= copurchase, edge weight = category similarity
fhr=open("amazon-books-copurchase.edgelist", 'rb')
copurchaseGraph=networkx.read_weighted_edgelist(fhr)
fhr.close()

In [3]:
print ("Looking for Recommendations for Customer Purchasing this Book:")
print ("--------------------------------------------------------------")
purchasedAsin = '0805047905'
print ("ASIN = ", purchasedAsin) 
print ("Title = ", amazonBooks.loc[purchasedAsin,'Title'])
print ("SalesRank = ", amazonBooks.loc[purchasedAsin,'SalesRank'])
print ("TotalReviews = ", amazonBooks.loc[purchasedAsin,'TotalReviews'])
print ("AvgRating = ", amazonBooks.loc[purchasedAsin,'AvgRating'])
print ("DegreeCentrality = ", amazonBooks.loc[purchasedAsin,'DegreeCentrality'])
print ("ClusteringCoeff = ", amazonBooks.loc[purchasedAsin,'ClusteringCoeff'])
    

Looking for Recommendations for Customer Purchasing this Book:
--------------------------------------------------------------
ASIN =  0805047905
Title =  Brown Bear, Brown Bear, What Do You See?
SalesRank =  171
TotalReviews =  172
AvgRating =  5.0
DegreeCentrality =  216
ClusteringCoeff =  0.01976744186046512


In [4]:
# Now let's look at the ego network associated with purchasedAsin in the
# copurchaseGraph - which is esentially comprised of all the books 
# that have been copurchased with this book in the past
# (1) Get the depth-1 ego network of purchasedAsin from copurchaseGraph,
#     and assign the resulting graph to purchasedAsinEgoGraph.

n=purchasedAsin
ego = networkx.ego_graph(copurchaseGraph,n, radius=1)
purchasedAsinEgoGraph = networkx.Graph(ego)

In [5]:
# Next, recall that the edge weights in the copurchaseGraph is a measure of
# the similarity between the books connected by the edge. So we can use the 
# island method to only retain those books that are highly simialr to the 
# purchasedAsin
# (2) Use the island method on purchasedAsinEgoGraph to only retain edges with 
#     threshold >= 0.5, and assign resulting graph to purchasedAsinEgoTrimGraph
threshold = 0.5
purchasedAsinEgoTrimGraph = networkx.Graph()
for f,t,e in purchasedAsinEgoGraph.edges(data=True):
    if e['weight'] >= threshold:
        purchasedAsinEgoTrimGraph.add_edge(f,t,weight=e['weight'])
        

In [6]:
# Next, recall that given the purchasedAsinEgoTrimGraph you constructed above, 
# you can get at the list of nodes connected to the purchasedAsin by a single 
# hop (called the neighbors of the purchasedAsin) 
# (3) Finding the list of neighbors of the purchasedAsin in the 
#     purchasedAsinEgoTrimGraph, and assign it to purchasedAsinNeighbors
purchasedAsinNeighbors = list(purchasedAsinEgoTrimGraph.neighbors(purchasedAsin))
purchasedAsinNeighbors = pd.DataFrame(purchasedAsinNeighbors)
purchasedAsinNeighbors.head()

,0
0,0152010661
1,1581170769
2,0688109942
3,0688148999
4,0152002650


In [7]:
# (4) Note that, given an asin, you can get at the metadata associated with  
#     it using amazonBooks (similar to lines 29-36 above).
#    This is a composite measure to make Top Five book 
#     recommendations based on one or more of the following metrics associated 
#     with nodes in purchasedAsinNeighbors: SalesRank, AvgRating, 
#     TotalReviews, DegreeCentrality, and ClusteringCoeff. Feel free to compute
#     and include other measures if you like.
#     Also, remember to transform the data appropriately using 
#     sklearn preprocessing so the composite measure isn't overwhelmed 
#     by measures which are on a higher scale.
#The metrics AvgRating, Clustering coefficient, Degree Centrality, Sales Rank and Total reviews were used to arrive at the
##Composite score. The steps followed can be broken down as follows:
#1)	Converted these columns into float datatype.
#2)	Did preprocessing of all these columns using Minmax scalar. Now, all these columns are between the range 0 to 1.
#3)	The lower the sales rank, the better a product is selling. So, transformed this column as SalesRank inverse which basically
##has the score of (1-SalesRank). 
#4)	Now, equal weightage was given to all these 5 metrics and arrived at Composite score. 
##Composite score is the average of AvgRating, Clustering coefficient, Degree Centrality, Sales Rank Inverse and Total reviews.
#5)	Sorted the table based on the Composite score and returned Top5 from that table.


amazonBooks=amazonBooks.reset_index()
purchasedAsinNeighbors['index']=purchasedAsinNeighbors[0]
purchasedAsinNeighbors=purchasedAsinNeighbors[['index']]

amazonBooks[['AvgRating', 'ClusteringCoeff','DegreeCentrality','SalesRank','TotalReviews']]= amazonBooks[['AvgRating', 'ClusteringCoeff','DegreeCentrality','SalesRank','TotalReviews']].astype('float64')
numericvars = ['AvgRating', 'ClusteringCoeff','DegreeCentrality','SalesRank','TotalReviews']
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
transformed_amazon_books = pd.DataFrame(mms.fit_transform(amazonBooks[numericvars]), columns=['mms_'+x for x in numericvars])
transformed_amazon_books = pd.concat([amazonBooks,transformed_amazon_books], axis=1)
transformed_amazon_books['SalesRank_inverse'] =1- transformed_amazon_books['mms_SalesRank']
transformed_amazon_books['Composite_score']=( transformed_amazon_books['mms_AvgRating']+transformed_amazon_books['mms_ClusteringCoeff']+transformed_amazon_books['mms_DegreeCentrality']+transformed_amazon_books['SalesRank_inverse']+transformed_amazon_books['mms_TotalReviews'])/5
transformed_amazon_books.head()


,index,AvgRating,Categories,ClusteringCoeff,DegreeCentrality,Id,SalesRank,Title,TotalReviews,mms_AvgRating,mms_ClusteringCoeff,mms_DegreeCentrality,mms_SalesRank,mms_TotalReviews,SalesRank_inverse,Composite_score
0,0827229534,5.0,christianity books subjects religion clergy sp...,0.464286,8.0,1,396585.0,Patterns of Preaching: A Sermon Sampler,2.0,1.0,0.464286,0.024691,0.104410,0.000361,0.895590,0.476986
1,0738700797,4.5,based books witchcraft subjects religion earth...,0.583333,9.0,2,168596.0,Candlemas: Feast of Flames,12.0,0.9,0.583333,0.027778,0.044387,0.002164,0.955613,0.493778
2,0486287785,5.0,hobbies books home garden subjects crafts general,0.000000,0.0,3,1270652.0,World War II Allied Fighter Planes Trading Cards,1.0,1.0,0.000000,0.000000,0.334527,0.000180,0.665473,0.333131
3,0842328327,4.0,reference life new bibles bible history study ...,0.600000,6.0,4,631289.0,Life Application Bible Commentary: 1 and 2 Tim...,1.0,0.8,0.600000,0.018519,0.166201,0.000180,0.833799,0.450500
4,1577943082,0.0,living christianity books prayerbooks subjects...,1.000000,4.0,5,455160.0,Prayers That Avail Much for Business: Executive,0.0,0.0,1.000000,0.012346,0.119831,0.000000,0.880169,0.378503


In [8]:
# Print Top 5 recommendations (ASIN, and associated Title, Sales Rank, 
# TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff)

  
def top5_recomm(purchasedAsinNeighbors,transformed_amazon_books):
    tmp=pd.merge(purchasedAsinNeighbors,transformed_amazon_books,on='index',how='left')
    df=tmp.sort_values('Composite_score',ascending=False).head(5)
    print("Top 5 Recommendations are:")
    df=df[['index','Title','SalesRank','TotalReviews','AvgRating','DegreeCentrality','ClusteringCoeff','Composite_score']]
    return df

top5_recomm(purchasedAsinNeighbors,transformed_amazon_books)

Top 5 Recommendations are:


,index,Title,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff,Composite_score
21,067167949X,Chicka Chicka Boom Boom,5537.0,89.0,5.0,4.0,1.0,0.605388
26,0399208534,The Very Hungry Caterpillar,2117.0,164.0,4.5,5.0,1.0,0.588890
14,0399237720,Very Hungry Caterpillar Giant Board Book and P...,112356.0,164.0,4.5,5.0,1.0,0.583086
23,0060235152,From Head to Toe,43438.0,22.0,5.0,5.0,0.9,0.581593
32,0399220496,The Very Hungry Caterpillar Book with Plush Ca...,181105.0,164.0,4.5,5.0,1.0,0.579466
